# ArrayManager Example Notebook

In [ ]:
# --- Imports ---
import pymif.microscope_manager as mm
import napari
import os
import numpy as np
import dask.array as da

In [ ]:
# Step 1: Create a synthetic 5D image (T, C, Z, Y, X)
t, c, z, y, x = 10, 2, 512, 2048, 2048
np_data = np.random.randint(0, 5000, size=(t, c, z, y, x), dtype=np.uint16)
dask_data = da.from_array(np_data, chunks=(1, 1, 1, 1024, 1024))

In [ ]:
# Step 2: Define metadata
metadata = {
            "size": [dask_data.shape],
            "scales": [(4.0, 0.65, 0.65)],
            "units": ("micrometer", "micrometer", "micrometer"),
            "time_increment": 1.0,
            "time_increment_unit": "s",
            "channel_names": ["Red", "Green"],
            "channel_colors": [16711680, 255],
            "dtype": "uint16",
            "axes": "tczyx",
        }

In [ ]:
# Step 3: Instantiate ArrayManager
dataset = mm.ArrayManager(dask_data, metadata)
print("Loaded data shape:", dataset.data[0].shape)

In [ ]:
# Step 4: Build pyramid (starting from base level)
dataset.build_pyramid(num_levels=3)
print("New pyramid levels:", [arr.shape for arr in dataset.data])

In [ ]:
# # Step 5 (optional): Visualize
# viewer = dataset.visualize()

In [ ]:
# Reorder channels
dataset.reorder_channels([1,0])
# Confirm channels reodered
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")

In [ ]:
# Change metadata
dataset.update_metadata({
    "channel_names":["Blue", "Red"],
    "time_increment_unit": "m",
})
# Confirm metadata changed
for i in dataset.metadata:
    print(f"{i.upper()}: {dataset.metadata[i]}")

In [ ]:
# Write the data to disk in serial but with some compression
import time
start = time.time()
dataset.to_zarr(
    "/g/mif/people/gritti/code/pymif_test_data/test_array_write_serial_GZIP.zarr",
    compressor = "GZIP",
    parallelize = False
)
print(time.time()-start)

In [ ]:
dataset_zarr = mm.ZarrManager("/g/mif/people/gritti/code/pymif_test_data/test_array_write_serial.zarr")

In [ ]:
# Check that writing happens in parallel
import time
start = time.time()
dataset_zarr.write(
    "/g/mif/people/gritti/code/pymif_test_data/test_array_write_resaved_parallel.zarr",
    compressor = None,
    parallelize = True
)
print(time.time()-start)